In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv("G:/Education/Machine Learning/ML_DATA_FILES/RNN Dataset Google Stocks/Google_Stock_Price_Train.csv")

In [3]:
train_set = df_train.iloc[:, 1:2].values

In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
sc = MinMaxScaler(feature_range=(0,1))

In [6]:
train_set_scaled = sc.fit_transform(train_set)

In [7]:
train_set_scaled.shape

(1258, 1)

### Creating a data structure with timesteps and 1 output

The use of time steps.
    * At each time T the rnn is going to look back to the stock prices between 60 days before time T and time T. 
    * Based on the 60 previous stock prices it will predict new output.
    * We can use as many time steps as possible and fixing to one number by doing trail & error method.

In [8]:
x_train = []
y_train = []
for i in range(60, 1258): # 60 be the starting point and 1258 be the end point as there are 1257 elements in the training data set # 
    x_train.append(train_set_scaled[i-60:i, 0]) # If i = 60 then 60-60=0 then train_set_scaled[0:60] will start from 0 to 60and the process continues#
    y_train.append(train_set_scaled[i,0])

In [9]:
x_train, y_train = np.array(x_train), np.array(y_train)

### Reshaping

We can find all the related docs of tensorflow keras in keras documentation. The belo code is imported from Keras Docs

In [10]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))

# Building RNN

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
regressor = Sequential()

### Adding first LSTM Layer & Some Dropout Regularisation

In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1],1))) # Need to mention the type of layer need to add #

In [ ]:
regressor.add(Dropout(0.2))

### Adding Second LSTM Layer & Dropout Regularisation

In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True)) # As this is second lstm layer there's no need of mentioning the input_shape. The input shape is considered from the first lstm layer#

In [ ]:
regressor.add(Dropout(0.2))

### Adding Third LSTM Layer & Dropout Regularisation

In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True))

In [ ]:
regressor.add(Dropout(0.2))

### Adding Fourth LSTM Layer & Dropout Regularisation

In [ ]:
regressor.add(LSTM(units = 50)) # As this is the final Lstm layer there's no need of mentioning return_sequences #

In [ ]:
regressor.add(Dropout(0.2))

### Adding the output layer

In [ ]:
regressor.add(Dense(units = 1)) # In order to connect all the layers in the tensorflow we need to use "DENSE" #

# Compiling RNN

In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN

In [ ]:
regressor.fit(x_train, y_train, epochs = 100, batch_size = 32)

# Predictions

### Predicting Real stock prices of 2017

In [ ]:
df_test = pd.read_csv("G:/Education/Machine Learning/ML_CSV_FILES/RNN Dataset Google Stocks/Google_Stock_Price_Test.csv")

In [ ]:
real_stock_price = df_test.iloc[:, 1:2].values

Open helps in considering all the columns and its variables in to the dataset.

For horizantal concating we use axis = 1

For vertical concating we use axis = 0

In [ ]:
df_total = pd.concat((df_train['Open'], df_test['Open']), axis = 0) 

As we are predicting the future values of the stock price we need inputs to predict.
    * As we have mentioned the considering of stock prices of previous 60 days in order to predict future. We need to provide the  date we need the price to be predicted.
    
    * The code will go through the data and considers 60 daya prior data.
    
    * 60 being the lower bound.

In [ ]:
inputs = df_total[len(df_total) - len(df_test) - 60:].values

In [ ]:
inputs = inputs.reshape(-1,1)

In [ ]:
inputs = sc.transform(inputs)

In [ ]:
x_test = []

In [ ]:
for i in range(60, 80): # 60 being the lower bound, and  in the test set we have 20 values which means stock prices of 20 days. So 60days + 20days in the test set = 80 #
    x_test.append(inputs[i-60:i, 0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1)) # 3D shape formatting #

In [ ]:
prediction = regressor.predict(x_test) 

In [ ]:
prediction = sc.inverse_transform(prediction)

# Visualisations

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(real_stock_price, color = 'blue', label = 'Real Stock Price')
plt.plot(prediction, color = 'red', label = 'Predicted Stock Price of Jan 2017')
plt.title("Comparison of Stock Prices")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()